In [1]:
from ModelClass import RegressionModel
from DatasetClass import Dataset, DatasetMass
import matplotlib.pyplot as plt
import numpy as np
import itertools
from src.helpers import make_filter_slice
import tensorflow as tf
import optuna 
from optuna_dashboard import run_server
import threading
#import sys
#import logging 


2025-03-26 22:42:27.295456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743025347.308738 3454435 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743025347.312781 3454435 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-26 22:42:27.327634: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
erik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*H125*.root"
patrik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*Ztt*.root"


In [ ]:
dataset = DatasetMass(file_paths=patrik_data)
dataset.load_data(file_name = "data")
#start_time = time.time()
#for batch in dataset.train_dataset.take(1):
#    print("Batch loaded in:", time.time() - start_time)
#train_batches = dataset.train_dataset.batch(32)
#for feature, target in train_batches.take(1):
#    print(f"Feature: {feature.shape}")
#    print(f"Target: {target.shape}")
#print(dataset.train_events)
#print(len(dataset.train_dataset))
#print(len(dataset.dev_dataset))
#print(len(dataset.val_dataset))
#This block of code iterates through the dataset and extracts the pt values of the labels and stores them in a list
#data = [labels.numpy()[0] for features, labels in dataset.train_dataset.take(100000)]

#plt.hist(data, bins=100, range=(50, 130), histtype='step', label='pt distribution', density=False)
#plt.legend(loc='upper right')
#plt.title('pt distribution of the dataset')
#plt.xlabel('pt')
#plt.ylabel('Number of events')
#plt.show()

#features, masses = next(iter(dataset.train_dataset.batch(1000)))

#print(features.shape)
#print(features)

#plt.hist(masses, range=(70, 130), bins=50)
#plt.show()



7799005
7799005


In [5]:
dataset.make_slices(n_slices=100)
slices = dataset.slices   

lorentz_mask = tf.constant(dataset.get_lorentz_mask())  # Shape [35] bool values
lorentz_indices = tf.squeeze(tf.where(lorentz_mask), axis=1) # [0 1 2 3 4 5 6 7 13 14 ...] 
n_vectors = tf.shape(lorentz_indices)[0] // 4 # number of 4-vectors
lorentz_indices_4d = tf.reshape(lorentz_indices, (n_vectors, 4))  # [n_vectors, 4]

@tf.function
def augment_lorentz(data, target):
    beta = tf.random.uniform(shape = (), minval=-0.98, maxval=0.98) # Shape ()
    #tf.print("used beta: ", beta)
    gamma = 1.0 / tf.sqrt(1.0 - beta**2) # Shape ()
    #tf.print("used gamma: ", gamma)
    
    for i in range(n_vectors):
        vec_indices = lorentz_indices_4d[i] # Shape (4,)
        pt = data[vec_indices[0]] # scalar values
        eta = data[vec_indices[1]]
        #tf.print("eta: ", eta)
        phi = data[vec_indices[2]]
        E = data[vec_indices[3]]
        #tf.print("E: ", E)
        #print(E.shape)
        
        # Convert to Cartesian coordinates
        #px = pt * tf.cos(phi)
        #py = pt * tf.sin(phi)
        pz = pt * tf.sinh(eta) # Shape ()
        #tf.print("pz: ", pz)

        E_prime = gamma * (E - beta * pz) # Shape ()
        #tf.print("E_prime: ", E_prime)
        pz_prime = gamma * (pz - beta * E) 
        
        epsilon = 1e-8
        eta_prime = tf.asinh(pz_prime / (pt + epsilon)) # Shape ()
        #tf.print("eta_prime: ", eta_prime)
        update_indices = tf.reshape(vec_indices, [-1, 1])  # Shape (4, 1)
        update_values = tf.stack([pt, eta_prime, phi, E_prime]) # Shape (4,)
        # Update the tensor
        data = tf.tensor_scatter_nd_update(
            data,
            indices=update_indices,
            updates=update_values
        )
        #print(data.shape)
        
    return data, target

n_events = 10000
new_dataset = tf.data.Dataset.sample_from_datasets([s.repeat() for s in slices], weights=[1.]*len(slices))
#orig_features, orig_masses = next(iter(new_dataset))
#print(orig_features)
#print(orig_masses)
new_dataset = new_dataset.take(n_events)
augmented_dataset = new_dataset.map(augment_lorentz)
#new_features, new_masses = next(iter(new_dataset))
#print(new_features)š
#print(new_masses)
#batch_dataset = augmented_dataset.batch(n_events)
#print(f"Number of events in batch_dataset: {len(augmented_dataset)}")
#features, masses = next(iter(batch_dataset))

#plt.hist(masses, range=(70, 130), bins=50)
#plt.show()

                                                                    

In [ ]:

lorentz_mask = tf.constant(dataset.get_lorentz_mask()) 
print(lorentz_mask)

features = tf.constant([
    5.56329613e+01, -3.16469967e-01, -9.03481603e-01, 6.74349565e-07,
    5.47039528e+01, -1.49002433e-01, 2.52283901e-01, 1.05658375e-01,
    1.67467535e-01, 1.15576553e+00, 1.16783524e+00, 1.38526611e+02,
    1.10336914e+02, 1.38102692e+02, 1.74350947e-01, 2.87319326e+00,
    1.55200872e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
    0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
    0.00000000e+00, 4.71827660e+01, 0.00000000e+00, -6.81111366e-02,
    -1.58858541e-02, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
    0.00000000e+00, 1.00000000e+00, 1.00000000e+00
], dtype=tf.float32)

label = tf.constant(94.92591, dtype=tf.float32)

dataset = tf.data.Dataset.from_tensors((features, label))
dataset = dataset.map(augment_lorentz)

new_features, new_label = next(iter(dataset))
print(new_features)
print(new_label)

In [ ]:
dataset.make_slices(n_slices=10)
slices = dataset.slices 
phi_mask = tf.constant(dataset.get_phi_mask())

@tf.function
def augment_phi(data, target):
    # generate random rotation angle
    angle = tf.random.uniform(shape=(tf.shape(data)[0],), minval=-np.pi, maxval=np.pi)
    #tf.print("angle: ", angle.shape)
    #tf.print("data: ", data.shape)
    # apply rotation
    data  = tf.where(phi_mask, data + angle, data)
    
    # normalize angles between -pi and pi
    data = tf.where(phi_mask, tf.math.atan2(tf.sin(data), tf.cos(data)), data)
    
    return data, target

# sample from the slices
n_events = 1000
new_dataset = tf.data.Dataset.sample_from_datasets([s.repeat() for s in slices], weights=[1.]*len(slices))
new_dataset = new_dataset.take(n_events)

# apply augmentation
new_dataset = new_dataset.map(augment_phi)
#aug_dataset = new_dataset.batch(n_events)


#print(f"Number of events in batch_dataset: {len(batch_dataset)}")
#features, masses = next(iter(batch_dataset))
#print(features.shape)
#print(features)

#plt.hist(masses, range=(70, 130), bins=50)
#plt.show()




In [ ]:
param_grid = {
    'batch_size': [3200, 6400],
    'learning_rate': [0.01, 0.1],
    'epochs': [5, 20, 30]
}

iterable = list(itertools.product(*param_grid.values()))
for params in iterable:
    model = RegressionModel(dataset=dataset, batch_size=params[0], initial_learning_rate=params[1], n_epochs=params[2])
    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model()
    model.plot_history()

In [ ]:
def objective(trial):

    model = RegressionModel(
                dataset,
                n_layers             = trial.suggest_int('n_layers', 2, 5),
                hidden_layer_size    = trial.suggest_int('hidden_layer_size', 512, 1024, step=128),
                initial_learning_rate= trial.suggest_float('initial_learning_rate', 1e-3, 1e-2, log=True),
                n_epochs             = trial.suggest_int('n_epochs', 10, 20, step=5),
                activation_function  = 'relu',
                batch_size           = trial.suggest_int('batch_size', 512, 1024,step=64),
                dropout_rate         = trial.suggest_float('dropout_rate', 0.1, 0.5),
                weight_decay         = trial.suggest_float('weight_decay', 1e-5, 1e-4, log=True)
            )

    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model()

    # Handle based on the number of metrics
    evaluation_results = model.model.evaluate(model.dev_batch, verbose=0)
    if isinstance(evaluation_results, list):
        val_loss = evaluation_results[0]
    else:
        val_loss = evaluation_results  # If only one value is returned, it's the loss

    return val_loss

study = optuna.create_study(storage="sqlite:///optuna.db", study_name="Higgs_analysis_mass_2", direction='minimize', load_if_exists=True)

def run_dashboard():
    server = run_server("sqlite:///optuna.db", host="0.0.0.0", port=8080)
    server.run()  # Run dashboard continuously in a separate thread

dashboard_thread = threading.Thread(target=run_dashboard)
dashboard_thread.daemon = True  # Ensures it stops when the script ends
dashboard_thread.start()

def run_optuna():
    study.optimize(objective, n_trials=100, n_jobs=1)

# Run Optuna in a separate thread so the dashboard can be accessed in real-time
optuna_thread = threading.Thread(target=run_optuna)
optuna_thread.start()

optuna_thread.join()  # Wait for Optuna to finish before printing results
print("Number of finished trials:", len(study.trials))
print("Best trial:", study.best_trial.params)



[I 2025-03-25 23:41:16,016] Using an existing study with name 'Higgs_analysis_mass_2' instead of creating a new one.
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.



Batching datasets...


2025-03-25 23:41:16.510918: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Building model...
Epoch 1/20


I0000 00:00:1742942477.678492 2786360 service.cc:148] XLA service 0x7f7c1c005ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742942477.678524 2786360 service.cc:156]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2025-03-25 23:41:17.711543: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742942477.828448 2786360 cuda_dnn.cc:529] Loaded cuDNN version 90300


...

12185/12185 ━━━━━━━━━━━━━━━━━━━━ 12s 924us/step - loss: 1592.9148 - mean_absolute_percentage_error: 101.2956 - mean_squared_error: 1592.9148 - val_loss: 707.6653 - val_mean_absolute_percentage_error: 7.4744 - val_mean_squared_error: 707.9465
Epoch 3/20
12185/12185 ━━━━━━━━━━━━━━━━━━━━ 100s 8ms/step - loss: 536.9959 - mean_absolute_percentage_error: 10.2609 - mean_squared_error: 536.9961 - val_loss: 681.8136 - val_mean_absolute_percentage_error: 7.7511 - val_mean_squared_error: 682.0729
Epoch 4/20
12185/12185 ━━━━━━━━━━━━━━━━━━━━ 11s 896us/step - loss: 1529.3429 - mean_absolute_percentage_error: 102.1909 - mean_squared_error: 1529.3429 - val_loss: 716.2120 - val_mean_absolute_percentage_error: 8.5863 - val_mean_squared_error: 716.4556
Epoch 5/20
 8609/12185 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - loss: 475.4726 - mean_absolute_percentage_error: 9.7364 - mean_squared_error: 475.4728

KeyboardInterrupt: 

 8610/12185 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - loss: 475.4728 - mean_absolute_percentage_error: 9.7364 - mean_squared_error: 475.4730

 9085/12185 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 475.6042 - mean_absolute_percentage_error: 9.7359 - mean_squared_error: 475.6045

[W 2025-03-25 23:46:08,653] Trial 50 failed with parameters: {'n_layers': 2, 'hidden_layer_size': 768, 'initial_learning_rate': 0.0034933707772084595, 'n_epochs': 20, 'batch_size': 640, 'dropout_rate': 0.39597475121024717, 'weight_decay': 9.481872484282561e-05} because of the following error: ZMQError('Socket operation on non-socket').
Traceback (most recent call last):
  File "/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_2786236/4236431915.py", line 19, in objective
    model.train_model()
  File "/home/ivanpa/HiggsAnalysisBc/ModelClass.py", line 138, in train_model
    history = self.model.fit(
  File "/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/ipykernel/iostream.py"